# Time Series Analysis Tool

This notebook exists to allow time series analysis of pixels or areas.

This notebook has been setup to work with table of contents and heading collapsing jupyter extensions. These are definitely worth using!

The output is an interactive matplotlib figure with two scenes from the area of interest rendered in true colour, along with scatter plots for the two main bands of interest for each scene, and an average spectral signature for each scene.

The main benefit to this visualistion though is the ability to see the the time series trend for a band or number of bands for a given pixel or area. This is one in a single chart, which also indicates the relative location of each of the scenes being viewed on the timeseries, and also shows some key threshold lines.

This particular notebook has been written to look at output from Peter's urban change detection algorithm, so if that algorithm hasn't been run over the study area, this code will break without some tweaks.

Written by Mike Barnes as part of his third graduate rotation, 2017.

## Imports

In [1]:
%matplotlib notebook
import os

import numpy as np
import pandas as pd
import xarray as xr

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

import gdal

import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual, IntSlider, FloatSlider, Dropdown
from IPython.display import display

from skimage import exposure
from scipy.signal import lfilter

import warnings

# Functions for Loading Data and Building the Xarray
This project built on some existing work by Peter Tan. An output from Peter's urban change detection algorithm is raster files with all the relevant NBAR (analysis ready satellite derived surface reflectance readings) data saved to the output directory. To speed the loading and analysis during this script, this notebook will use those exisitng files if they are available. Otherwise it will load the data from the Digital Earth Australia archive.

## function: checkForLocalFiles

In [2]:
def checkForLocalFiles(study_area):
    rootdir = os.listdir('../')
    if study_area in rootdir:
        return True
    else:
        return False

## function: getData

In [3]:
def getData(study_area):
    # if the study area is a string, and is accessible locally, load it
    if isinstance(study_area, str):
        if checkForLocalFiles(study_area):
            data = getLocalData(study_area)
            print('Done')
            return data
    # if the study area is a string and is on the list, load it
        else:
            data = DCLoadName(study_area)
    # if the study area is a list of coordinates, use them to load the data
    elif isinstance(study_area, list) and len(study_area) == 4:
        data = DCLoad(study_area)
        
    # if the study area isn't loaded locally, transfrom the DC originated xarray into "my" format  
    if not checkForLocalFiles(study_area):
        data = transformXarrayToCustomStyle(data)
        print('Done')
        return data
    else:
        print('Data Loading Error')

## function: DCLoadName
This function is a wrapper for the DCLoad function, that allows previously used study areas to be easily restudied
by easily loading exactly the same area of interest (AOI).

In [4]:
def DCLoadName(study_area):   
    if study_area == 'mtbarker':
        lat_min = -35.05
        lat_max = -35.08
        lon_min = 138.85
        lon_max = 138.895  
    elif study_area == 'swmelb':
        lat_min = -37.879
        lat_max = -37.91
        lon_min = 144.705
        lon_max = 144.76  
    elif study_area == 'gunghalin':
        lat_min = -35.18
        lat_max = -35.21
        lon_min = 149.14
        lon_max = 149.17
    elif study_area == 'goldengrove': 
        lat_min = -34.77
        lat_max = -34.8
        lon_min = 138.66
        lon_max = 138.73
    elif study_area == 'molonglo':
        lat_min = -35.3
        lat_max = -35.33
        lon_min = 149.015
        lon_max = 149.06
    elif study_area == 'nperth':
        lat_min = -31.686
        lat_max = -31.73
        lon_min = 115.79
        lon_max = 115.813
    elif study_area == 'swbris':
        lat_min = -27.66
        lat_max = -27.7 
        lon_min = 152.877
        lon_max = 152.93
    elif study_area == 'swsyd':
        lat_min = -33.993
        lat_max = -34.04
        lon_min = 150.715 
        lon_max = 150.78
    
    return DCLoad([lat_min, lat_max, lon_min, lon_max])

## function: DCLoad
This function is a variation of a datacube query supplied by Erin Telfer.

In [5]:
def DCLoad(study_area):
    # to time how long the load takes
    start = datetime.datetime.now()
    print('Loading data') 
    print('Load Started At: ' + str(start))
    
    # define temporal range 
    start_of_epoch = '1987-01-01'
    end_of_epoch =  '2017-10-31'

    # define bands of interest
    bands_of_interest = ['blue', 'green', 'red', 
                         'nir', 'swir1', 'swir2']

    # Landsat sensors of interest are defined
    sensors = ['ls8', 'ls7', 'ls5'] 

    # unpack input parameter
    lat_min, lat_max, lon_min, lon_max = study_area    

    print('Bounding box: ' + str(lat_min) + ' S, ' + str(lon_min) +
          ' E to ' + str(lat_max) + ' S, ' + str(lon_max) + ' E' )
    print('Epoch: ' + start_of_epoch + ' to ' + end_of_epoch)
    print('Sensors: ' + str(sensors))
    print('Bands of Interest: ' + str(bands_of_interest))

    # create query
    query = {'time': (start_of_epoch, end_of_epoch),}
    query['x'] = (lon_min, lon_max)
    query['y'] = (lat_max, lat_min)
    query['crs'] = 'EPSG:4326'

    #Create cloud mask. This will define which pixel quality (PQ) artefacts are removed from the results.
    # It should be noted the "land_sea" code will remove all ocean/sea pixels.
    mask_components = {'cloud_acca':'no_cloud',
    'cloud_shadow_acca' :'no_cloud_shadow',
    'cloud_shadow_fmask' : 'no_cloud_shadow',
    'cloud_fmask' :'no_cloud',
    'blue_saturated' : False,
    'green_saturated' : False,
    'red_saturated' : False,
    'nir_saturated' : False,
    'swir1_saturated' : False,
    'swir2_saturated' : False,
    'contiguous':True,
    'land_sea': 'land'}

    # Connect to DataCube
    dc = datacube.Datacube(app='Urban Change Detection')
    
    # Data for each Landsat sensor is retrieved and saved in a dict for concatenation
    sensor_clean = {}
    
    for sensor in sensors:
        # Load the NBAR and corresponding PQ
        sensor_nbar = dc.load(product= sensor+'_nbar_albers', group_by='solar_day', 
                              measurements = bands_of_interest,  **query)
        sensor_pq = dc.load(product= sensor+'_pq_albers', group_by='solar_day', 
                            fuse_func=ga_pq_fuser, **query)

        # Retrieve the projection information before masking/sorting
        crs = sensor_nbar.crs
        crswkt = sensor_nbar.crs.wkt
        affine = sensor_nbar.affine        

        # Combing the pq so it is a single 
        sensor_all = xr.auto_combine([sensor_pq,sensor_nbar])
        sensor_clean[sensor] = sensor_all

        print('Loaded %s' % sensor) 

    print('Concatenating')
    nbar_clean = xr.concat(sensor_clean.values(), 'time')
    nbar_clean = nbar_clean.sortby('time')
    nbar_clean.attrs['crs'] = crs
    nbar_clean.attrs['affin|e'] = affine    

    print ('Load and Xarray build complete')
    print('Process took ' + str(datetime.datetime.now() - start))
    
    # return xarray
    return nbar_clean

## function: getLocalData

In [6]:
def getLocalData(study_area):
    """A quick helper function to load the output files from Peter's code for the given location.
    It returns and Xarray of the landsat data for that study area."""
    # build a list of all files in the directory (ie the folder for that location)
    location = '../' + study_area + '/'
    files = os.listdir(location)

    print('Loading data from: ' + location)
    
    # build a list of all the NBAR*.img file names and which bands they represent
    NBARfiles = []
    bands = []
    for file in files:
        if file[-4::] == '.img' and file[0:4] == 'NBAR':
            NBARfiles.append(file)
            bands.append(file.split('NBAR_')[1].split('.img')[0])

    # open all the .img files with NBAR in the name, convert to numpy array, swap axes so order is (x, y, t)
    # and save to dict
    raw_data = {}
    for i in range(len(NBARfiles)):
        raw_data[bands[i]] = gdal.Open(location + NBARfiles[i]).ReadAsArray().swapaxes(0,2)
#     num_scenes = len(raw_data['red'][0][0])   # delete this?

    # build a list of all the dates represented by each band in the NBAR files
    # reuse the list of NBAR file names, but this time access the .hdr file
    in_dates = False
    dates = []
    for line in open(location + NBARfiles[0].split('.img')[0] + '.hdr'):
        if line[0] == '}':
            continue
        if in_dates:
            dates.append(line.split(',')[0].strip())
        if line[0:10] == 'band names':
            in_dates = True

    # save list of satellite originated bands
    sat_bands = bands.copy()

    # add the yet to be calculated derivative bands to the overall bands list
    bands += ['cloud_mask', 'evi','ndvi','albedo']

    # building the Xarray
    # define the size for the numpy array that will hold all the data for conversion into XArray
    x = len(raw_data['red'])
    y = len(raw_data['red'][0])
    t = len(raw_data['red'][0][0])
    n = len(bands)

    # create an empty numpy array of the correct size
    alldata = np.zeros((x, y, t, n), dtype=np.float32)

    # populate the numpy array with the satellite data
    # turn all no data NBAR values to NaNs
    for i in range(len(sat_bands)):
        alldata[:,:,:,i] = raw_data[sat_bands[i]]
        alldata[:,:,:,i][alldata[:,:,:,i] == -999] = np.nan

    # convert the numpy array into an xarray, with appropriate lables, and axes names
    data = xr.DataArray(alldata, coords = {'x':range(x), 'y':range(y), 'date':dates, 'band':bands},
                 dims=['x', 'y', 'date', 'band'])
    
    # import cloudmask and add to xarray
    cloudmask = gdal.Open(location + 'tsmask.img').ReadAsArray().swapaxes(0,2)
    data.loc[:,:,:,'cloud_mask'] = cloudmask
    
    return data

## function: transformXarrayToCustomStyle

In [7]:
def transformXarrayToCustomStyle(data_new):
    # loop through and extract all the values into a list of 3D numpy arrays
    bands = ['blue','green','red','nir','swir1','swir2','pixelquality']
    all_data = []
    for band in bands:
        all_data.append(data_new.variables[band].values.swapaxes(2,0).astype(np.float32))
        
    # making some placeholders for yet to be derived "bands" - specific to the TSViewer
    extra_bands = ['evi','ndvi','albedo']
    for band in extra_bands:
        d = np.zeroslike(all_data[0])
        d[d == 0] = np.nan
        all_data.append(d)

    # replace 'pixelquality' with 'cloud_mask'  
    bands[bands.index('pixelquality')] = 'cloud_mask'

    # stack the list of 3D arrays along a 4th dimension
    data_flipped = np.stack(all_data, axis = -1)
    # set all bad NBAR values to np.nan
    data_flipped[data_flipped == -999] = np.nan
    # set all good pixels to 0 based on PQ indicator
    # see https://www.sciencedirect.com/science/article/pii/S0034425717301086 for PQ description
    data_flipped[data_flipped[:,:,:,-1] == 16383] = 0

    # save the new shape for use in defining the Xarray
    new_shape = data_flipped.shape

    # build a list of all the dates (as strings)
    dates = []
    for time in range(len(data_new.time)):
        dates.append(np.datetime_as_string(data_new.time[time].values)[0:10])

    # make the full list of bands
    all_bands = bands + extra_bands
    
    # assemble the new Xarray
    newdatafixed = xr.DataArray(data_flipped, coords = {'x': range(new_shape[0]), 'y': range(new_shape[1]),
                                                        'date': dates, 'band': all_bands}, dims=['x','y','date','band'])
    return newdatafixed

## function: getChangeResults

In [8]:
def getChangeResults(study_area):
    # open the results of the change algorithm, and format for eay plotting
    change = gdal.Open('../' + study_area + '/change_time.img').ReadAsArray()

    return change

## function: getChangeMask

In [9]:
def getChangeMask(study_area):
    # open the results of the change algorithm, and format for eay plotting
    change = gdal.Open('../' + study_area + '/change_time.img').ReadAsArray()
    change_flat_mask = change.copy()
    
    # remove the dates, so you have a mask for change or no change
    change_flat_mask[change_flat_mask != 0] = 1
    change_flat_mask[change_flat_mask == 0] = np.nan
    
    return change_flat_mask

## function: populateExtraBands

In [10]:
def populateExtraBands(study_area):
    # calculate NDVI "band"
    a = (data.loc[:,:,:,'nir'] - data.loc[:,:,:,'red'])
    b = (data.loc[:,:,:,'nir'] + data.loc[:,:,:,'red'])
    b_pos = b.where(b.values > 0)
    data.loc[:,:,:,'ndvi'] = np.nan
    data.loc[:,:,:,'ndvi'] = a/b_pos

    # calculate EVI "band"
    g  = 2.5
    c1 = 6
    c2 = 7.5
    l  = 1
    data.loc[:,:,:,'evi'] = g * ((a/10000) /
        ((data.loc[:,:,:,'nir']/10000) + (c1 * (data.loc[:,:,:,'red']/10000)) - (c2 * (data.loc[:,:,:,'blue']/10000)) + l))

    # calculate average albedo "band"
    sat_bands = ['blue','green','red','nir','swir1','swir2']
    for band in sat_bands:
        data.loc[:,:,:,'albedo'] += data.loc[:,:,:,band]
    data.loc[:,:,:,'albedo'] = data.loc[:,:,:,'albedo'] / (6 * 1000)

# Defining Functions

## Data Preparation and Manipulation Functions

### getTimeSeries

In [11]:
def getTimeSeries(xpos, ypos, ts_bands, apply_cloud_mask = True):
    """
    This function retrives the time series data for a given pixel and band.
    It then applies the cloud/shadow mask (if requested) to the results and to the dates for each scene.
    Finally, it returns the good pixels for each of the requested bands as a multi column pandas dataframe,
    with the date formatted as a datetime object and set as in the index
    
    Possible flaw with logic by only masking pixels when they are clouded, not entire query area if any or some
    is partially clouded. Need to explore this thought further
    """       
    if isinstance(xpos, list) and len(xpos) == 1:
        xpos.append(xpos[0] + 1)
    elif not isinstance(xpos, list) and is_number(xpos):
        xpos = [int(xpos), xpos + 1]
        
    if isinstance(ypos, list) and len(ypos) == 1:
        ypos.append(ypos[0] + 1)
    elif not isinstance(ypos, list) and is_number(ypos):
        ypos = [int(ypos), ypos + 1]
    
    # if looking for an area and not masking for cloud, do this
    if not apply_cloud_mask:
        ts_data = pd.DataFrame(data[xpos[0]:xpos[1], ypos[0]:ypos[1],:].sel(band = ts_bands).mean(dim=['x','y'], skipna = True).to_pandas())
        
    # if looking for an area and masking for cloud, do this
    else:
        ts_data = pd.DataFrame(data[xpos[0]:xpos[1], ypos[0]:ypos[1], :].sel(band = ts_bands).where(
            data[xpos[0]:xpos[1], ypos[0]:ypos[1], :].sel(band ='cloud_mask') == 0).mean(dim=['x','y'], skipna = True).to_pandas())
    
    ts_data.index = pd.to_datetime(ts_data.index) 
    ts_data.columns = [ts_bands]
    return ts_data

### filterNoise

In [12]:
def filterNoise(df):
    """
    This function applies a scipy.signal.lfilter IIR filter to the time series data to smooth out
    some of the spikes that are visible in this timeseries for each of the columns in the supplied
    pandas dataframe.
    Peters C++ code used a different filter, so this function may be replaced or deprecated in the
    future
    """
    # set up required parameters for the function
    n = 15                 # bigger n means more smoothing
    b = [1.0 / n] * n
    a = 1      
    for col in df.columns:
        df[col] = lfilter(b, a, df[col])
    return df

### calcRollingAvg

In [13]:
def calcRollingAvg(df, window = '365D', min_periods = 6):
    """
    The function uses the pandas rolling average function to calculate the yearly rolling
    average of the bands. It returns the dataframe it was given with the column values modified
    by the rolling average function.
    The window size and minimum number of periods for the average to be considered valid are optional
    parameters
    Peter's code used either 1 or 2 years windows depending on frequency of valid observations - this might
    be included in the future
    """
    for col in df.columns:
        df[col] = df[col].rolling(window = window, min_periods = min_periods).mean()
    return df

### calcYearDiff

In [14]:
def calcYearDiff(df, years = 1):
    """
    Returns the difference between a timeseries and the timeseries at some point in the future.
    It does this by joining a time adjusted series onto the original data and interpolating any missing
    dates, and then cleaning up the NaN mess resulting.
    """
    # convert years to days
    days = years * 365
    
    # duplicate data
    df2 = df.copy()

    # reduce the index by a year, essentially moving the average for the following year back one
    # this might need a bit of tweaking to deal with leap years...?
    df2.index = np.array(df2.index) - np.timedelta64(days, 'D')
    
    # building lists of column names that will be needed
    cols = list(df.columns)
    newcols = []
    interpcols = []
    diffcols = []
    for col in cols:
        newcols.append(col + '-1y')
        interpcols.append(col + '-1y_interp')
        diffcols.append(col + '_diff')
    
    # rename df2 columns to avoid clashes in the merge
    df2.columns = newcols

    # SQL style outer join (ie all data is kept, NaNs for blank values)
    df = pd.merge(df, df2, left_index=True, right_index=True, how='outer')
 
    # interpolate the timeseries values in the future and calculate the difference from that date
    for i in range(len(cols)):
        df[interpcols[i]] = df[newcols[i]].interpolate(method='time')
        df[diffcols[i]] = df[interpcols[i]] - df[cols[i]]

    # tidying up unwanted rows and columns
    # can do dropna because if there was no original data, the difference calc result is NaN
    df = df.dropna(subset=diffcols)
    df = df.drop(cols + newcols + interpcols, axis = 1)
    return df

### gatherAndPrepTSData

In [30]:
def gatherAndPrepTSData(xpos,ypos,ts_bands):
    """
    This function takes the x and y coordinates of interest, along with the ts_bands dict.
    It pulls the required data from the xarray, and applies the various processing steps specified in the dict
    It returns a pandas dataframe of all the processed data, with a date index, and each column named as
    specified in the dict.
    """
    dfs = []
    cols = []
    # ts_bands is the very important dict, it specifies the name of the result, the band the result is built
    # off, and which of the processing steps will be performed on that band data
    for TS in ts_bands:
        df = getTimeSeries(xpos, ypos, TS['band'], apply_cloud_mask = TS['cloud_mask'])
        # drop rows with clouds masked
        df = df.dropna(axis = 0, how = 'all')
        
        # drop any duplicate dates
        df = df[~df.index.duplicated(keep = 'first')]

        # filter noise
        if TS['smoothed']:
            df = filterNoise(df)

        # calculate rolling average
        if TS['RollingAve']:
            df = calcRollingAvg(df)
            # drop rows with no rolling average values
            df = df.dropna(axis = 0, how = 'all')

        # calculate difference between TS value and a period into the future
        if TS['TSDiff']:
            df = calcYearDiff(df)
        
        dfs.append(df)
        cols.append(TS['name'])
    dfs = pd.concat(dfs, axis = 1)
    dfs.columns = cols
    return dfs

## Helper functions for plotting and other

### define_TS_YAxisRange

In [16]:
def define_TS_YAxisRange(data, DiffTS = False, n_stddevs = 0.5):
    """
    This function defines the y axis range for a timeseries plot of a given band (supplied as a pandas 
    single column dataframe or series, aka data. It calculates the standard deviation of the data, and 
    returns a list with the lowest and highest values for the axes based on the number of standard deviations
    away from the mean (which can be varied through the second parameter as desired)
    """
    if DiffTS:
        mag = data.std(skipna = True) * n_stddevs
        return [-mag, mag]
    else:
        mean = data.mean(skipna = True)
        std = data.std(skipna = True)
        low = mean - (n_stddevs * std)
        hi = mean + (n_stddevs * std)
        return [low,hi]

### is_number

In [17]:
def is_number(s):
    """
    A quick helper function pinched from stack overflow to test if a variable is a float or int
    """
    try:
        float(s)
        return True
    except ValueError:
        return False
    

### getSceneFilteredIndex

In [18]:
def getSceneFilteredIndex(scene_num, df):
    """
    This function find the X axis location needed for a given scene to be displayed on the Time Series View.
    This is used draw the lines on the timeseries that represent the scenes being viewed.
    If the scene being viewed is part of that pixel's timeseries, the line returned is a solid line.
    If the scene being viewed is not part of that pixel's timeseries (due to cloud masking), the
    line returned is a dashed line.
    The line is returned as a number, linestyle pair, for plugging into the matplotlib.axvline function
    """
    # find the date of the scene_number from the list of all dates before clouds mask application
    linedate = pd.to_datetime(dates[scene_num])
    
    # for the simple case, when the scene is part of the pixel's time series
    if linedate in df.index:
        nearest_num = df.index.get_loc(linedate)
        return [nearest_num, '-']
    
    # when the pixel is not part of the time series and is more recent than the most recent valid
    # scene, use the last valid scenes index
    elif linedate > df.index[-1]:
        nearest_num = df.index.get_loc(df.index[-1])
        return [nearest_num, '--']
    
    # when the schene is not part of the pixel's time series, draw the line at the first valid scene's
    # index AFTER the scene in question
    else:   
        nearest_num = df.index.get_loc(df[df.index > linedate].index[0])
        return [nearest_num, '--']

### getChangePeriodTimeRange

In [19]:
def getChangePeriodTimeRange(df, daterange):
    """
    This function changes the calculated change date from Peter's algorithm and converts it
    to the equivalent range of row numbers for the given timeseries (supplied as a pandas dataframe).
    The change date calculated is converted to the start and end dates of the quarter, and this is 
    turned into a datetime object, and used in boolean masking to define the first and last row
    of that interval
    """
    # flaw with the logic here, if someone passes a daterange with more than 2 entries, only the first
    # and last are used. So it works, as long as its used well.
    for i in range(len(daterange)):
        year = str(int(daterange[i]))    
        # generate the string equivalents of the start and end dates for the quarter
        if daterange[i] % 1 == 0.125:
            start = '-01-01'
            end = '-03-31'
        if daterange[i] % 1 == 0.375:
            start = '-04-01'
            end = '-06-30'
        if daterange[i] % 1 == 0.625:
            start = '-07-01'
            end = '-09-30'
        if daterange[i] % 1 == 0.875:
            start = '-10-01'
            end = '-12-31'
        if i == 0:
            startdate = pd.to_datetime(year + start) # turn strings into datetime objects
        else:
            enddate = pd.to_datetime(year + end)    
    
    # calculdate the start and end date index of the range bounded by start and end datetimes
    start_idx = df[(df.index >= startdate) & (df.index <= enddate)].index[0]
    end_idx = df[(df.index >= startdate) & (df.index <= enddate)].index[-1]    
  
    # get the row number of the start and end indexes
    start_row = df.index.get_loc(start_idx)    
    end_row = df.index.get_loc(end_idx)
    
    # return result as a list, ready to directly be passed to axis.set_ylim()
    return [start_row, end_row]

### getChangeDate

In [20]:
def getChangeDate(xpos, ypos):
    """
    This function returns the max and min change date for a given range of pixels based on the
    output of the algorithm. It works if the range is only a single pixel.
    It returns a string if there was no change detected in that area.
    """
    # select the pixels in the area of interest
    pix = change[ypos[0]:ypos[1], xpos[0]:xpos[1]]
    # mask for > 0
    pix = pix[pix > 0]
    # if there are no pixels > 0, break things
    if len(pix) == 0:
        return "No Change"
    # find hte max and min and return them as a list
    mini = pix.min()
    maxi = pix.max()
    return [mini, maxi]

## Plotting Functions

### drawScene

In [21]:
def drawScene(scene_num, change_trans, xpos, ypos):
    """
    This function draws a landsat scene from the data in true colour, and overlays the results of the
    change detection algorithm. The scene to be displayed is determined by the scene_num parameter, while the
    transparency (alpha) of the change results is dictacted by the change_trans parameter (0 = transparent,
    1 = totally opaque). It formats the axes appropriately and returns the axes to the caller.
    It also displays the bounding box of the pixels being analysed.
    """
    # colour map included incase of need to display false colour or other in the future
    # could change this to an ordereddict and remove the RGB list created below...?
    colourmap = {'R':'red', 'G':'green', 'B':'blue'}
    
    # define the current colour map to display the change results raster properly
    current_cmap = matplotlib.cm.get_cmap('Reds_r')
    current_cmap.set_under('k', alpha=0.0)
    current_cmap.set_over('r', alpha=1.0)
    current_cmap.set_bad('k', alpha=0.0)  
    
    # combine the data for the 3 bands to be displayed into a single numpy array
    h = data.shape[1]
    w = data.shape[0]
    RGB = ['R','G','B']
    
    # create array to store the RGB info in, and fill by looping through the colourmap variable
    # note the .T at the end, because the data array is setup as a (x,y,t), but imshow works (y,x)
    rawimg = np.zeros((h, w, 3), dtype=np.float32)
    for i in range(len(RGB)):     
        rawimg[:,:,i] = data[:,:,scene_num].sel(band=colourmap[RGB[i]]).T
        
    # equalizing for all bands together
    # goal is to make is human interpretable
    img_toshow = exposure.equalize_hist(rawimg, mask = np.isfinite(rawimg))    

    # displaying the results and formatting the axes etc
    plt.imshow(img_toshow)
    ax = plt.gca()
    ax.set_title('True Colour Landsat Scene, taken\n' + dates[scene_num] + ', over ' + study_area)
    ax.imshow(change_flat_mask, alpha = change_trans, interpolation='none', cmap = current_cmap, clim = [0.5, 0.6])
    
    # plot up the displayed pixel, or draw a box around the queried pixels
    # logic re . vs box could be improved here
    height = abs(ypos[1] - ypos[0])
    width = abs(xpos[1] - xpos[0])
    if height > 2 and width > 2:
        rect = matplotlib.patches.Rectangle((xpos[0],ypos[0]), abs(xpos[1] - xpos[0]), abs(ypos[1] - ypos[0]),
                                            color = 'blue', linestyle = '-', fill = False, alpha = change_trans)
        ax.add_patch(rect)
    else:
        ax.plot((xpos[0] + xpos[1])/2, (ypos[0] + ypos[1])/2, color = 'b', marker = '.', alpha = change_trans)
    return ax

### drawSpectrum

In [22]:
def drawSpectrum(xpos, ypos, scene_num):
    """
    This function draws the spectral signature for a selected range of pixels in a selected scene.
    It gathers the data together, averages the band values, and returns them plotted on nicely labelled
    axes, which are returned to the caller
    """
    # include Landsat-8 bands incase of future work
    bandorder = {'ultrablue':1, 'blue':2, 'green':3, 'red':4, 'nir':5, 'cirrus':6, 'swir1':7,
                 'swir2':8, 'thermal':9,'panchromatic':10}
    sat_bands = ['blue','green','red','nir','swir1','swir2']
    
    # do lots of funkiness to pull the data together and order it so the data are displayed
    # in order of increase wavelength
    xvals = []
    yvals = []
    for band in sat_bands:
        xvals.append(bandorder[band])
        yvals.append(float(data[xpos[0]:xpos[1],ypos[0]:ypos[1],scene_num].sel(band=band).mean(dim=['x','y']).values))
    yvals_sorted = [y for x, y in sorted(zip(xvals,yvals))]
    xlabels_sorted = [y for x, y in sorted(zip(xvals, sat_bands))]
    xvals_sorted = range(len(yvals))
    
    # plot the data, and make it look nice
    plt.plot(xvals_sorted, yvals_sorted)
    ax = plt.gca()
    ax.set_title('Band Values for Selected Area')
    ax.set_ylim([0,6000])
    ax.set_ylabel('Average NBAR Value')
    ax.set_xticks(xvals_sorted)
    ax.set_xticklabels(xlabels_sorted)
    ax.set_xlabel('Band')
    return ax  

### drawScatter

In [23]:
def drawScatter(scene_num, ts_bands, xpos, ypos):
    """
    This function draws a 2D scatter plot of the scene, based on the two bands of interest (specified by
    x_axis and y_axis). It returns the scatter plot to the caller.
    This basically assumes that the first and second bands in ts_bands are the two of most interest to the
    user, so ts_bands needs to be structured appropriately.
    The longer term goal would be to have dropdown boxes that would allow the user to select which band from the
    xarray they would like for each axis.
    """
    # set the axis bands
    x_axis = ts_bands[0]['band']
    y_axis = ts_bands[1]['band']
    
    # get the 1D array of the values for each axis
    x = data[:,:,scene_num].sel(band = x_axis).values
    y = data[:,:,scene_num].sel(band = y_axis).values
    
    # pull out the values from the bounding box selected
    sel_x = data[xpos[0]:xpos[1], ypos[0]:ypos[1], scene_num].sel(band = x_axis).values
    #create a full size array full of NaN
    sel_x_all = np.full(x.shape, np.nan)
    #set the corresponding area of the NaN array to the values from the bounding box
    sel_x_all[xpos[0]:xpos[1],ypos[0]:ypos[1]] = sel_x
    
    # repeat
    sel_y = data[xpos[0]:xpos[1], ypos[0]:ypos[1], scene_num].sel(band = y_axis).values
    sel_y_all = np.zeros_like(y)
    sel_y_all[xpos[0]:xpos[1],ypos[0]:ypos[1]] = sel_y
    
    
    # build a mask of where the values are both valid (ie not NaN)
    mask = np.isfinite(x) & np.isfinite(y)
    mask2 = np.isfinite(sel_x_all) * np.isfinite(sel_y_all)
    
    # make plot, label axes and return
    plt.plot(x[mask], y[mask],'.', color = 'blue', label = 'All Points')
    plt.plot(sel_x_all[mask2], sel_y_all[mask2],'.', color = 'red', label = 'Selected Points')
    ax = plt.gca()
    ax.set_xlabel(x_axis)
    ax.set_xlim([0,1])
    ax.set_ylabel(y_axis)
    ax.set_ylim([0,5])
    ax.legend()
    return ax

### drawTimeSeries

In [24]:
def drawTimeSeries(xpos,ypos,ts_bands,left_scene_num = 0, right_scene_num = 1):
    """
    This function draws the time series for a given pixel and band (or list of 2 bands).
    It calls the timeSeries function to gather the data and strip out the cloud, and then
    formats the axes appropriately for up to 2 bands and returns them to the caller.
    It also indicates where on the time axes each of the scenes being viewed sits, and displays them as
    either solid or dashed lines depending on if the pixels in question are cloud masked or not.
    It also displays as a yellow box the time range over which the change detection algorithm has indicated
    change has occured.
    Finally, it shows as horizontal lines the thresholds used for defining change in Peter's algorithm
    """ 
    # get the data, filter cloud, filter noise, calculate rolling average, drop NaN rows, calc difference
    df = gatherAndPrepTSData(xpos, ypos, ts_bands)
    # can't remember what this is for, I think it can be deleted now
    yearDiff = True

    # see if/when that pixel changed
    changedate = getChangeDate(xpos, ypos)
    changed = False
    if isinstance(changedate, list):     #changedate only returns a list if there was change
        changed = True
        # if it did change, get the row numbers of the first and last scene taken during the change quarter
        # to draw the box aronud the potential change period
        changetimerange = getChangePeriodTimeRange(df, changedate)
    
    # only plot up the first 4 bands requested (this might be changed soon)
    cols = list(df.columns)
    colours = ['green', 'red', 'blue', 'orange', 'black']
    axs = []
    n_axes = min(len(cols),5)

    # setup some variables for use during the plotting
    leftline = getSceneFilteredIndex(left_scene_num, df)    
    rightline = getSceneFilteredIndex(right_scene_num, df)    
    albedo_thresh = 0.04
    evi_thresh = -0.05
    resid_evi = 0.18    
    
    #start plotting
    plt.plot(range(len(df)), df[cols[0]], color=colours[0], label = cols[0])
    ax0 = plt.gca()
    pos1 = ax0.get_position()
    pos2 = pos1
#     pos2 = [pos1.x0 + (0.03 * n_axes), pos1.y0,  pos1.width / (1 + (0.1 * (n_axes - 1))), pos1.height] 
    ax0.set_position(pos2)
    ax0.spines['right'].set_color(colours[0])

    # draw the scene viewer lines
    ax0.axvline(x = leftline[0], linestyle = leftline[1]) 
    ax0.axvline(x = rightline[0], linestyle = rightline[1])
    
    # draw a zero line (x axis) for the TSDiff lines
    ax0.axhline(y = 0, color = 'black', linestyle = '-')
     
    # if there was change, draw the yellow rectangle    
    if changed:
        ax0.axvspan(changetimerange[0],changetimerange[1], color = 'gold', alpha = 0.7)
#         yrange = define_TS_YAxisRange(df[cols[0]], yearDiff, 2)
#         ax0.text(changetimerange[1], yrange[1] - ((yrange[1] - yrange[0]) / 10), changedate)
        
    # twin the x axis for creating a date axis on the top
    ax_date = ax0.twiny()
    # plot some random data with 100% transparency to make the axis appear
    ax_date.plot(df.index, df[cols[0]], alpha = 0)
    # set the axis to overlap with the original axis
    ax_date.set_position(pos2)
    
    # create a title that specifies the area being displayed
    readable_coords = 'x = ' + str(xpos[0]) + ':' + str(xpos[1]) + ', y = ' + str(ypos[0]) + ':' + str(ypos[1])
    ax0.set_title('Time Series values for pixels ' + readable_coords, y = 0)

    # for all the subsequent axes (not the first column)
    for i in range(1, n_axes):
        # twin the original axes
        axn = ax0.twinx()
        axn.set_position(pos2)
        # set the spines and the position and the colour
        axn.spines['right'].set_position(('axes', - (0.1 * i)))
        axn.spines['right'].set_color(colours[i])
        # plot the data, coloured accordingly
        axn.plot(range(len(df)), df[cols[i]], color=colours[i], label = cols[i])
        # add to the axs list of axes
        axs.append(axn)
    
    # the original axis to the start of the list
    axs = [ax0] + axs
    
    for i, ax in enumerate(axs):
        # for each axis, calculate the appropriate yaxis range, and set the xaxis range
        yrange = define_TS_YAxisRange(df[cols[i]], ts_bands[i]['TSDiff'], 2)
        ax.set_ylim(yrange)
        ax.set_xlim([0, len(df)])
        
        if i == 0:
            # if its the first axis, move the yaxis label
            ax.set_ylabel(cols[i], color=colours[i], labelpad = -10)
            # getting specific to the use case now, must be EVIDiff as first column
            ax.axhline(y = evi_thresh, color = colours[i], linestyle = '--')
            # print the EVI value for the RHS displayed scene
            after_evi = float(data[xpos[0]:xpos[1], ypos[0]:ypos[1], right_scene_num].sel(band = 'evi').mean().values)
            ax0.text(rightline[0], yrange[1] - ((yrange[1] - yrange[0]) / 10), 'EVI = ' + str(round(after_evi, 3)))       
        else:
            ax.set_ylabel(cols[i], color=colours[i], labelpad = -40)
        # more use case specificness
        if i == 1:
            ax.axhline(y = albedo_thresh, color = colours[i], linestyle = '--')
        if i == 2:
            ax.axhline(y = resid_evi, color = colours[i], linestyle = '--')
    return axs

### drawAllSubplots

In [25]:
def drawAllSubplots(left_scene_num, right_scene_num, change_trans, ts_bands, xpos, ypos, BBoxWidth, BBoxHeight):
    """
    This function defines what plots will be shown, and in what order. This function is called
    by the the analyse function and also during the onclick event.
    It returns nothing, but draws the axes on command
    """
    # the xpos and ypos here come from the mouse click event, ie are a pixel coordinate.
    # need to convert this to a list for use as a bounding box
     
    if BBoxWidth == 1:
        # if only a pixel is wanted, the list will be len == 1
        xrange = [xpos, xpos + 1]
    else:
        # split the witdth in half and take either side
        # might need to confirm edge cases here...?
        xrange = [int(xpos - BBoxWidth/2), int(xpos + BBoxWidth/2)]
    
    if BBoxHeight == 1:
        yrange = [ypos, ypos + 1]
    else:
        yrange = [int(ypos - BBoxHeight/2), int(ypos + BBoxHeight/2)]
    
    # might need some changing in the future regarding formating, plot spacing etc.   
    ax1 = plt.subplot2grid([7,4],[0,0], rowspan = 2, colspan = 2)
    ax1.clear()
    ax1 = drawScene(left_scene_num, change_trans, xrange, yrange)    

    ax2 = plt.subplot2grid([7,4],[0,2], rowspan = 2, colspan = 2)
    ax2.clear()
    ax2 = drawScene(right_scene_num, change_trans, xrange, yrange)  

    ax3 = plt.subplot2grid([7,4],[2,0], rowspan = 2, colspan = 4)
    ax3.clear()
    ax3 = drawTimeSeries(xrange, yrange, ts_bands, left_scene_num, right_scene_num)

    ax4 = plt.subplot2grid([7,4],[4,0], rowspan = 2, colspan = 2)
    ax4.clear()
    ax4 = drawScatter(left_scene_num, ts_bands, xrange, yrange)    

    ax5 = plt.subplot2grid([7,4],[4,2], rowspan = 2, colspan = 2)
    ax5.clear()
    ax5 = drawScatter(right_scene_num, ts_bands, xrange, yrange) 
    
    ax6 = plt.subplot2grid([7,4],[6,0], rowspan = 1, colspan = 2)        
    ax6.clear()
    ax6 = drawSpectrum(xrange, yrange, left_scene_num)                       

    ax7 = plt.subplot2grid([7,4],[6,2], rowspan = 1, colspan = 2)
    ax7.clear()
    ax7 = drawSpectrum(xrange, yrange, right_scene_num)

    plt.tight_layout()
    plt.draw()

### analysis

In [26]:
global xpos
global ypos
xpos = 0
ypos = 0
  
def analysis(left_scene_num, right_scene_num, change_trans, ts_bands, BBoxWidth = 1, BBoxHeight = 1):

    def onclick(event):
        # defining what to do on a click event
        
        # I don't understand why this need to be declared global again, but it breaks without these lines
        global xpos
        global ypos
        # need to cast to int as result is a float, and can't index a list with a float
        xpos = int(event.xdata)
        ypos = int(event.ydata)
        drawAllSubplots(left_scene_num, right_scene_num, change_trans, ts_bands, xpos, ypos, BBoxWidth, BBoxHeight)
    
    # control the figure size
    fig = plt.figure(figsize=[10,15])
    plt.subplots_adjust(hspace = 0.6)
    
    # draw the figure
    drawAllSubplots(left_scene_num, right_scene_num, change_trans, ts_bands, xpos, ypos, BBoxWidth, BBoxHeight)
    #connect the click event action to the figure
    cid = fig.canvas.mpl_connect('button_press_event', onclick)

# Running it

## Chosing the Location

In [27]:
# create a study area drop down list and display it
study_areas = ['mtbarker', 'swmelb', 'gunghalin', 'goldengrove', 'molonglo', 'nperth', 'swbris', 'swsyd'] 
study_area_dd = Dropdown(options=study_areas, value = study_areas[0], description='Study Area', disabled = False)
display(study_area_dd)

Dropdown(description='Study Area', options=('mtbarker', 'swmelb', 'gunghalin', 'goldengrove', 'molonglo', 'nperth', 'swbris', 'swsyd'), value='mtbarker')

In [28]:
# importing the data getting everything ready for the visualisation
study_area = study_area_dd.value
data = getData(study_area)
populateExtraBands(study_area)
max_x, max_y, num_scenes, _ = data.shape
dates = list(data.date.values)
change = getChangeResults(study_area)
change_flat_mask = getChangeMask(study_area)

Loading data from: ../swmelb/
Done


/g/data/v10/public/modules/agdc-py3-env/20171214/envs/agdc/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in greater
  """


In [31]:
# TS_BANDS is the key datastructure for the whole setup.
# it is a list of dicts, each dict being the details required for the timeseries data gathering and preparation
# the first 2 items in the list will have their bands used as the x and y axis for the scatter plot

# *** dict keys and their use***
# name = the name of the dataseries on the resultant plot
# band = the band on which the analysis is based, this is used to directly access the xarray, so it must be identical
    # to the band in the xarray. 
# cloud_mask = Boolean as to if you want to use the cloud mask
# smoothed = Boolean as to if you want to use the smoothing filter
# RollingAve = Boolean as to if you want to use an annual rolling average rather than a raw/smoothed reading
# TSDiff = Boolean as to if you want to use the difference between that reading and the one 12 months ago.

ts_bands = [
    {'name':'evi_diff', 'band':'evi', 'cloud_mask':True, 'smoothed':True, 'RollingAve':True, 'TSDiff':True},
    {'name':'albedo_diff', 'band':'albedo', 'cloud_mask':True, 'smoothed':True, 'RollingAve':True, 'TSDiff':True},
    {'name':'evi_raw', 'band':'evi', 'cloud_mask':True, 'smoothed':True, 'RollingAve':False, 'TSDiff':False}]

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    interact(analysis,
             left_scene_num = IntSlider(value = 1, min = 0, max = num_scenes -1,
                                        description = "Left Scene Number"),
             right_scene_num = IntSlider(value = num_scenes -1, min = 0, max = num_scenes -1,
                                         description = "Right Scene Number"),
             change_trans = FloatSlider(value = 0.6, min = 0, max = 1, description = "Change Mask Transparency"),
             ts_bands = fixed(ts_bands),
            BBoxWidth = IntSlider(value = 1, min = 1, max = max_x, description = "BBox Width"),
            BBoxHeight = IntSlider(value = 1, min = 1, max = max_y, description = "BBox Height"))

interactive(children=(IntSlider(value=1, description='Left Scene Number', max=1322), IntSlider(value=1322, description='Right Scene Number', max=1322), FloatSlider(value=0.6, description='Change Mask Transparency', max=1.0), IntSlider(value=1, description='BBox Width', max=208, min=1), IntSlider(value=1, description='BBox Height', max=156, min=1), Output()), _dom_classes=('widget-interact',))